In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import matplotlib.pyplot as plt
import math
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
ca_data = pd.read_csv('/kaggle/input/youtube-new/CAvideos.csv')
ca_data.head()

In [ ]:
channel_dataset = pd.read_csv('../input/youtube-channels-100000/channels.csv')
channel_dataset.head()
channel_dataset.info()


# Data Cleaning

In [ ]:
with open('/kaggle/input/youtube-new/CA_category_id.json') as f:
    categories = json.load(f)
categories_map = {}
for category in categories['items']:
    categories_map[int(category['id'])] = category['snippet']['title']
ca_data['category_id'] = ca_data['category_id'].map(categories_map)
ca_data.head()

In [ ]:
print('info', ca_data.info())
print('Views stats', ca_data['views'].describe())
print('Comment stats', ca_data['comment_count'].describe())
print('Like stats', ca_data['likes'].describe())
ca_data.isnull().sum()

In [ ]:
channel_dataset = channel_dataset.rename(columns={'title':'channel_title', 'category_name': 'channel_category_name'})
new_ca_data= ca_data.merge(channel_dataset,how='left', on='channel_title')


In [ ]:
print('missing values percentage', new_ca_data['followers'].isnull().sum()/len(new_ca_data))
new_ca_data.info()

In [ ]:
ca_data['description'] = ca_data['description'].fillna(value='None',inplace=True)

In [ ]:
ca_data['publish_time'] = pd.to_datetime(ca_data['publish_time'], format = '%Y-%m-%dT%H:%M:%S.%fZ')
ca_data['publish_month'] = ca_data['publish_time'].dt.month
ca_data['publish_hour'] = ca_data['publish_time'].dt.hour

In [ ]:
ca_data['like_percentage'] = (ca_data['likes'] / ca_data['views']) * 100
ca_data['dislike_percentage'] = (ca_data['dislikes']/ca_data['views']) * 100
ca_data.loc[ca_data['likes'] >= ca_data['dislikes'],'more_likes'] = 1
ca_data.loc[ca_data['likes'] < ca_data['dislikes'],'more_likes'] = 0


In [ ]:
def get_title_length(title):
    return len(title)
def get_number_of_tags(string_of_tags):
    tags = string_of_tags.split('|')
    tags_no_empty_strings = []
    for tag in tags:
        if tag != '':
            tags_no_empty_strings.append(tag)
    return len(tags_no_empty_strings)

ca_data['title_length'] = ca_data.apply(lambda row: get_title_length(row['title']), axis=1)
ca_data['number_of_tags'] = ca_data.apply(lambda row: get_number_of_tags(row['tags']), axis=1)

In [ ]:
def contains_all_caps_word(title):
    words = title.split(' ')
    for word in words:
        if word.isupper():
            return True 
print(contains_all_caps_word('HEY how are you'))
    

# Visualizations

In [ ]:
f,ax = plt.subplots(3,1, figsize=(20,10))
views_by_categories_plot = sns.barplot(x=ca_data['category_id'], y=ca_data['views'], ax=ax[0])
views_by_categories_plot.set_title('Number of Views Per Category') 
likes_by_categories_plot = sns.barplot(x=ca_data['category_id'], y=ca_data['likes'], ax=ax[1])
likes_by_categories_plot.set_title('Number of Likes Per Category')     
comment_count_by_categories_plot = sns.barplot(x=ca_data['category_id'], y=ca_data['comment_count'], ax=ax[2])
comment_count_by_categories_plot.set_title('Number of Comments Per Category')  
plt.tight_layout()

In [ ]:
plt.figure(figsize=(16,6))
correlation_matrix_heatmap = sns.heatmap(ca_data[['views', 'likes', 'comment_count','dislikes', 'title_length', 'number_of_tags']].corr(),vmin=-1,vmax=1, cmap='YlGnBu', annot=True)
correlation_matrix_heatmap.set_title('Correlation Heatmap')

In [ ]:
f,ax = plt.subplots(2,1, figsize=(16,6))
views_by_hour = sns.lineplot(x=ca_data['publish_hour'], y=ca_data['views'],data=ca_data,marker='o', ax=ax[0])
views_by_hour.set_title('Views per Publish Hour')
views_by_month= sns.lineplot(x=ca_data['publish_month'], y=ca_data['views'],data=ca_data,marker='o', ax=ax[1])
views_by_month.set_title('Views per Publish Month')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(16,6))
feeling_barplot = sns.barplot(x=ca_data['category_id'], y=ca_data['views'], hue=ca_data['more_likes'], data=ca_data)
feeling_barplot.set_xticklabels(feeling_barplot.get_xticklabels(),rotation=30)
feeling_barplot.set_title('Views Per Category Per Feeling')
plt.tight_layout()

# Linear Regression

Here are some observations from our exploration that will inform the way we build our model:
1. Movies and Music historically get ma higher number of views than other categories. Including category as a predictor is a good idea 
2. Likes and Comment count are highly correlated. Since linear regression assumes multicolinearity it could be a good idea to remove one of them
3. The standard deviation on month 10 is quite high so it's unreasonable to draw an assumption that there is any statistical significance to videos published in this month getting many views
4. Over the few months the trending data was collected, we saw a high amount of views for videos published at 10 AM. There is little standard deviation on this high figure. This relationship with our target variable may mean we can use publish hour as a predictor
5. There is a relationship between likes and categories. Does that mean that one must be removed since we're assuming independance?
6. The title length and number of tags have a very low correlation with our target variable so it might not be worth including for our model

# Considerations

1. Theres a concept in machine learning called data leakage:


In [ ]:
ca_data = ca_data.drop(['comment_count'], axis=1)

In [ ]:
def get_title_length(title):
    return len(title)
def get_number_of_tags(string_of_tags):
    tags = string_of_tags.split('|')
    tags_no_empty_strings = []
    for tag in tags:
        if tag != '':
            tags_no_empty_strings.append(tag)
    return len(tags_no_empty_strings)
ca_data = ca_data.drop(['video_id', 'channel_title', 'description', 'likes', 'dislikes', 'thumbnail_link', 'comment_count', 'video_error_or_removed'],axis=1)
#Convert boolean valus to integers
ca_data = ca_data.astype({'comments_disabled': 'int', 'ratings_disabled': 'int'})
ca_data['publish_time'] = ca_data.apply(lambda row: get_publish_hour(row['publish_time']), axis=1)
ca_data = ca_data.rename(columns={'publish_time':'publish_hour'})
ca_data = pd.get_dummies(ca_data,prefix='category', columns=['category_id'])
ca_data = pd.get_dummies(ca_data,prefix='hour', columns=['publish_hour'])

ca_data['title_length'] = ca_data.apply(lambda row: get_title_length(row['title']), axis=1)
ca_data = ca_data.drop(columns=['title'])
ca_data['number_of_tags'] = ca_data.apply(lambda row: get_number_of_tags(row['tags']), axis=1)
ca_data = ca_data.drop(['tags', 'trending_date'],axis=1)


In [ ]:
y = ca_data['views']
X = ca_data[ca_data.columns.difference(['views'])]

In [ ]:
kfold = model_selection.KFold(n_splits=10, random_state=7, shuffle=True)
model = LinearRegression()
scoring = 'neg_mean_squared_error'
results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring)
print("MSE: %.3f (%.3f)" % (results.mean(), results.std()))